<a href="https://colab.research.google.com/github/mahii6991/transfer_learning_bert/blob/main/fine_tuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Helper function

In [5]:
 !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-26 12:17:29--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-26 12:17:29 (96.9 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [6]:
# export TORCH_USE_CUDA_DSA=1

In [7]:
# #here we will be coding a feature based , or working on the feature based model and understanding its accuracy with respect to our dataset
# from helper_functions import unzip_data, calculate_results
# unzip_data('/content/train_test.zip')

# Loading Dataset

In [8]:
import pandas as pd
df_train = pd.read_csv('/content/train.csv')
df_train.head()

,Unnamed: 0,sentences,Subject,Object,relations,relations_id
0,0,While each of the 80 high-potential start-ups ...,Niall McEvoy,Enterprise Ireland,managerOf,0
1,3,"Altada. First on the list is [E1]Altada[/E1], ...",Altada,Dublin,locatedAt,1
2,5,Fellow co-founder Niamh Parker serves as the c...,Allan,Trinity College Dublin,employedBy,2
3,6,"Boumdless. [E2]Boundless[/E2], said McEvoy, is...",Dee Coakley,Boundless,managerOf,0
4,8,GuardYoo. [E2]GuardYoo[/E2] is another HPSU ba...,Cork,GuardYoo,locatedAt,1


In [9]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.0 MB/s eta 0:00:00


In [10]:
#improting the training file into the system
training = pd.read_csv("/content/train.csv")
#improting the testing file after getting the zip file
testing = pd.read_csv("/content/test.csv")

* Creating the testing and training  and validation files in it. In order to create the validation file we have splitted the testing file and created a 20% subset of it in order to create the validation file.

In [11]:
#need to create the training file and the testing file
X_train = training['sentences']
y_train = training['relations_id']

#creating the testing file of the model
X_test = testing['sentences']
y_test = testing['relations_id']

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the training file
df_train = pd.read_csv("train.csv")

# Split the training data into train and validation sets
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)

# Save the validation data to a CSV file
val_df.to_csv("val.csv", index=False)


In this code the first 11 layers of the bert model is freezed and only the last layer is used , this is a very basic code to fine tune the bert model and will see how more changes can be done in the future.

In [14]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.6 MB/s eta 0:00:00


* When the file is created in the form of the dataloaders , the best way is to create a dictionary of the dataset , we have use the functio called as the load_dataset function which creates the dictionary of the data which contains the data splits and the associated features with the dataset.

In [15]:
from datasets import load_dataset

# Load the dataset
annotated_dataset = load_dataset("csv", data_files={
    "train": "train.csv",
    "validation": "val.csv",
    "test": "test.csv",
})

# Access the train, validation, and test splits
train_dataset = annotated_dataset["train"]
val_dataset = annotated_dataset["validation"]
test_dataset = annotated_dataset["test"]


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ff62d3c3c35b0ccf/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

* so basically our dataset is stored in the form of the dictionary with a key as the features and rest of the columns as the value.

In [16]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'sentences', 'Subject', 'Object', 'relations', 'relations_id'],
    num_rows: 911
})

In [17]:
val_dataset

Dataset({
    features: ['Unnamed: 0', 'sentences', 'Subject', 'Object', 'relations', 'relations_id'],
    num_rows: 183
})

In [18]:
test_dataset

Dataset({
    features: ['Unnamed: 0', 'sentences', 'Subject', 'Object', 'relations', 'relations_id'],
    num_rows: 228
})

* In the code snippet you provided, the AutoTokenizer.from_pretrained("distilbert-base-uncased") line creates an instance of the AutoTokenizer class and loads the tokenizer for the "distilbert-base-uncased" model. The tokenizer is responsible for converting raw text into numerical representations that can be understood by the model.

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


* here the target variable is tokenized with the added function of the padding and trucation.

In [20]:
# special_tokens_dict = {'additional_special_tokens': ['[E1]', '[/E1]', '[E2]', '[/E2]']}
# tokenizer.add_special_tokens(special_tokens_dict)

In [21]:
def tokenize_text(batch):
    return tokenizer(batch["sentences"], truncation=True, padding=True)

In [22]:
orca_tokenized = annotated_dataset.map(tokenize_text, batched=True, batch_size=None)

Map:   0%|          | 0/911 [00:00<?, ? examples/s]

Map:   0%|          | 0/183 [00:00<?, ? examples/s]

Map:   0%|          | 0/228 [00:00<?, ? examples/s]

In [23]:
orca_tokenized['train']

Dataset({
    features: ['Unnamed: 0', 'sentences', 'Subject', 'Object', 'relations', 'relations_id', 'input_ids', 'attention_mask'],
    num_rows: 911
})

In [24]:
#orca_tokenized['test']['sentences']

In [25]:
orca_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "relations_id"])

In [26]:
orca_tokenized['train']['input_ids'].shape

torch.Size([911, 227])

In [27]:
orca_tokenized['train']['attention_mask'].shape

torch.Size([911, 227])

In [28]:
from torch.utils.data import DataLoader, Dataset


class annotated_dataset(Dataset):
    def __init__(self, dataset_dict, partition_key="train"):
        self.partition = dataset_dict[partition_key]

    def __getitem__(self, index):
        return self.partition[index]

    def __len__(self):
        return self.partition.num_rows

In [29]:
train_dataset = annotated_dataset(orca_tokenized, partition_key="train")
val_dataset = annotated_dataset(orca_tokenized, partition_key="validation")
test_dataset = annotated_dataset(orca_tokenized, partition_key="test")

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=12,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=12,
    num_workers=2
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=12,
    num_workers=2
)

# Unfreezing last layers

In [30]:
# from transformers import AutoModelForSequenceClassification


# model = AutoModelForSequenceClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=4)

In [31]:
# for param in model.parameters():
#     param.requires_grad = False

In [32]:
# model

# Unfreezing last-layers - 2

In [33]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

# Unfreeze only the last two layers
total_layers = len(model.distilbert.transformer.layer)  # Get the total number of layers
unfrozen_layers = total_layers - 10  # Specify the number of layers to unfreeze

# Set requires_grad to True for the parameters in the unfrozen layers
for name, param in model.named_parameters():
    if any(layer in name for layer in [f"distilbert.transformer.layer.{i}" for i in range(unfrozen_layers, total_layers)]):
        param.requires_grad = True
    else:
        param.requires_grad = False


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.

In [34]:
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

# Print the model architecture
print(model)

# Check the requires_grad status of each parameter
for name, param in model.named_parameters():
    print(f"Parameter: {name}, Requires Grad: {param.requires_grad}")

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

# BertForSequenceClassification

In [35]:
# from transformers import BertForSequenceClassification

In [36]:
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# # Unfreeze the last two transformer layers
# for name, param in model.bert.encoder.layer[-2:].named_parameters():
#     param.requires_grad = True


In [37]:




# # Unfreeze the last two transformer layers
# for name, param in model.bert.encoder.layer[-2:].named_parameters():
#     param.requires_grad = True

In [38]:
# for param in model.pre_classifier.parameters():
#     param.requires_grad = True

# for param in model.classifier.parameters():
#     param.requires_grad = True

# Installing lightning

In [39]:
pip install lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [40]:
import lightning as L
import torch
import torchmetrics


class CustomLightningModule(L.LightningModule):
    def __init__(self, model, learning_rate=2e-5):
        super().__init__()

        self.learning_rate = learning_rate
        self.model = model

        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=4)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=4)

    def forward(self, input_ids, attention_mask, labels):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["relations_id"])
        self.log("train_loss", outputs["loss"])
        return outputs["loss"]  # this is passed to the optimizer for training

    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["relations_id"])
        self.log("val_loss", outputs["loss"], prog_bar=True)

        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self.val_acc(predicted_labels, batch["relations_id"])
        self.log("val_acc", self.val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["relations_id"])

        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self.test_acc(predicted_labels, batch["relations_id"])
        self.log("accuracy", self.test_acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


lightning_model = CustomLightningModule(model)

In [41]:

from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger


callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )  # save top 1 model
]
logger = CSVLogger(save_dir="logs/", name="my-model")

In [42]:
trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="gpu",
    precision="16-mixed",
    devices=[0],
    logger=logger,
    log_every_n_steps=10,
)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [43]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Rest of your code that uses CUDA


In [44]:
# set TORCH_USE_CUDA_DSA=1


In [45]:
# import torch

# torch.cuda.empty_cache()


In [46]:
import time
start = time.time()

trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name     | Type                                | Params
-----------------------------------------------------------------
0 | model    | DistilBertForSequenceClassification | 67.0 M
1 | val_acc  | MulticlassAccuracy                  | 0     
2 | test_acc | MulticlassAccuracy                  | 0     
-----------------------------------------------------------------
42.5 M    Trainable params
24.4 M    Non-trainable params
67.0 M    Total params
267.826   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name     | Type                                | Params
-----------------------------------------------------------------
0 | model    | DistilBertForSequenceClassification | 67.0 M
1 | val_acc  | MulticlassAccuracy                  | 0     
2 | test_acc | MulticlassAccuracy                  | 0     
------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


Time elapsed 0.91 min


# Training Accuracy

In [47]:
trainer.test(lightning_model, dataloaders=train_loader, ckpt_path="best")

INFO: Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:480: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9099889993667603     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.9099889993667603}]

In [48]:
trainer.test(lightning_model, dataloaders=val_loader, ckpt_path="best")

INFO: Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9016393423080444     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.9016393423080444}]

In [49]:
trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best")

INFO: Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=228.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.7850877046585083     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.7850877046585083}]